In [18]:
from grape import Analysis as an
from grape import controlField as cf
from grape import propagation as prop
from grape import config as cfg
from grape import rf
from plotly import express as px
import numpy as np

# Set simulation parameters

In [ ]:
# Global parameters
global_parameters = cfg.Global(
    N=2**9,
    T_s=5e-3, # Duration in seconds
    nucleus='H'
)

# Define different isochromat populations
population_1 = cfg.Population(
    application=global_parameters.application,
    B1p = np.linspace(0.3,1.54,21),  # B1p factor
    CS = 2 * np.pi * np.linspace(-2000, 2000, 101),  # Chemical shift in rad/s
)

# Set up the problem geometry
geometry_parameters = cfg.Geometry(
    application     = global_parameters.application,
    sliceThickness_mm=0,
    nb_stopBand=0,
    nb_transBand=0,
    nb_in_slice=0,
)

# Load the RF pulse

In [20]:
# Load the RF pulse
sech = rf.sech(global_parameters.t_s, mu=4.5, beta=1374, max_w1=26e-6*global_parameters.gamma)
# Create w1x control field
sech_wx = cf.ControlField(global_parameters=global_parameters, initial_temp_value=np.real(sech))
# Create w1y control field
sech_wy = cf.ControlField(global_parameters=global_parameters, initial_temp_value=np.imag(sech))
# Create slice selection gradient control field
sech_gz = cf.ControlField(global_parameters=global_parameters, initial_temp_value=np.zeros_like(sech))

# Define the global control field
sech_cf = {'w1x': sech_wx,
           'w1y': sech_wy,
           'gz': sech_gz}

# Propagate

In [21]:
isoList = prop.generate_isoList(population_list=[population_1], geo_parameters=geometry_parameters, global_parameters=global_parameters)
isoList_sech = prop.propagate_par(control_fields=sech_cf, isoList=isoList, global_parameters=global_parameters)

# Plots

In [22]:
# Plot control fields
an.plot_w1_field(control_fields=sech_cf, global_parameters=global_parameters, mode='mp', export=False)

# Plot final longitudinal magnetization
df_finalMagn_analysis = an.plot_final_magn(isoList=isoList_sech, plot=True, x='cs', y='M_z', color='B1p')